# Predspracovanie snímok z Lizard datasetu

**Autor: Bc. Ivan Vykopal**

Tento notebook obsahuje predsrpacovanie snímok predovšetkým z Lizard datasetu, pričoms a zameriavame na vytovreni výsekov o veľkosti 512 x 512 pixelov.

In [ ]:
import glob
import math
import os
import json

In [ ]:
def create_patches(file, root, directory, patch_size=512, check_size=False):
    
    file_name = file.replace(root,'').replace('.jpg', '').replace('.png', '').replace('.tif', '')
    img = cv.imread(file)
    # get image width and height
    size_y, size_x, channels = img.shape
    
    json = {
        "name": file.replace(root,''),
        "height": size_y,
        "width": size_x,
        "channels": channels,
        "patches": []
    }

    # count number of patches on every axis
    x_patches = int(math.ceil(size_x / patch_size))
    y_patches = int(math.ceil(size_y / patch_size))

    # iterate over patches
    for y_patch in range(y_patches):# riadok
        for x_patch in range(x_patches): #stlpec
            # get x and y coors of the start of the patch
            x = x_patch * patch_size
            y = y_patch * patch_size
            json['patches'].append((y_patch, x_patch))

            # check patch width and height
            width = size_x - x if (x + patch_size > size_x) else patch_size
            height = size_y - y if (y + patch_size > size_y) else patch_size
            
            if check_size:
                patch = np.full((patch_size, patch_size, channels), 255)
                patch[0:height, 0:width, :] = img[y: y + height, x:x + width, :]
            else:
                patch = img[y: y + height, x:x + width, :]

            # save image patch, file name contain x and y value of patch
            output_name = os.path.join(
                directory, file_name + '_' + str(y_patch) + '_' + str(x_patch) + '.tif')
            cv.imwrite(output_name, patch)
        
    return json

In [ ]:
source = ['lizard-data\\images\\']
destination = ['lizard-data\\patches']
types = ['train', 'test']

for i in range(len(source)):
    final_json = {
        "images": [],
        "patch_size": 512
    }
    print('Solving ' + str(i) + ' index of ' + str(len(source)))
    files = glob.glob(source[i] + '*')
    for file in files:
        img_json = create_patches(file, source[i], destination[i], patch_size=final_json['patch_size'], check_size=True)
        final_json['images'].append(img_json)

    json_string = json.dumps(final_json)
    with open('images-' + types[i] + '.json', 'w') as outfile:
        outfile.write(json_string)
